In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer

from keras.backend.tensorflow_backend import set_session
from keras.callbacks import Callback, LambdaCallback
from keras.datasets import cifar10, mnist
from keras.applications import MobileNet, imagenet_utils
from keras.optimizers import SGD, Adam

from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.models import Model
from keras import backend as K

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
set_session(sess)

In [ ]:
class OneCycleScheduler(Callback):
    def __init__(self, epochs, max_lr, steps_per_epoch, moms = (0.95, 0.85), div_factor = 25, start_pct = 0.3):
        # initialize the instance variables
        self.max_lr = max_lr
        self.moms = moms
        self.div_factor = div_factor
        self.st1_epochs = int(np.floor(epochs * start_pct))
        self.st2_epochs = epochs - self.st1_epochs
        self.st1_steps = self.st1_epochs * steps_per_epoch
        self.st2_steps = self.st2_epochs * steps_per_epoch
        self.history = {"lrs" : [], "moms" : []}
    
    def __annealing_cos(self, start, end, pct):
        "Cosine anneal from `start` to `end` as pct goes from 0.0 to 1.0."

        cos_out = np.cos(np.pi * pct) + 1    
        return end + (start - end) / 2 * cos_out
    
    def on_train_begin(self, logs = None):
        # initialize the necessary variables
        self.steps_so_far = 0         
    
    def on_batch_begin(self, batch, logs = None):
        # increment the step count         
        self.steps_so_far += 1
        
        # check to determine the training phase
        if self.steps_so_far <= self.st1_steps:
            # calculate the new learning rate             
            new_lr = self.__annealing_cos(self.max_lr / self.div_factor, 
                                          self.max_lr, 
                                          self.steps_so_far / self.st1_steps)
            
            # calculate the new momentum
            new_mom = self.__annealing_cos(self.moms[0],
                                          self.moms[1],
                                          self.steps_so_far / self.st1_steps)
            
            # set the new learning rate and momentum
            K.set_value(self.model.optimizer.lr, new_lr)
            K.set_value(self.model.optimizer.momentum, new_mom)

        else:
            # calculate the new learning rate             
            new_lr = self.__annealing_cos(self.max_lr, 
                                          self.max_lr / self.div_factor, 
                                          (self.steps_so_far - self.st1_steps) / self.st2_steps)
            
            # calculate the new momentum
            new_mom = self.__annealing_cos(self.moms[1],
                                           self.moms[0],
                                           (self.steps_so_far - self.st1_steps) / self.st2_steps)
            
            # set the new learning rate and momentum
            K.set_value(self.model.optimizer.lr, new_lr)
            K.set_value(self.model.optimizer.momentum, new_mom)
            
        # update the history attribute
        self.history["lrs"].append(new_lr)
        self.history["moms"].append(new_mom)

In [ ]:
# initialize the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# scale the images into [0, 1]
x_train = x_train.astype(np.float32) * (255.0 / x_train.max())
x_test = x_test.astype(np.float32) * (255.0 / x_test.max())

# preprocess the images
x_train = imagenet_utils.preprocess_input(x_train, mode = "tf")
x_test = imagenet_utils.preprocess_input(x_test, mode = "tf")

# convert the labels from integers into vectors
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [ ]:
class MobileNetClassifier:
    @staticmethod
    def build(base_model, classes):
        #  GlobalAveragePooling
        x = GlobalAveragePooling2D()(base_model.output)
        
        # FC => RELU block
        x = Dense(256, kernel_initializer = "he_normal")(x)
        x = Activation("relu")(x)
        
        # Softmax classifier
        x = Dense(classes, kernel_initializer = "he_normal")(x)
        x = Activation("softmax")(x)
        
        # return the constructed model architecture    
        return Model(inputs = base_model.input, outputs = x)

In [ ]:
# initialize the base model
base_model = MobileNet(include_top = False, weights = "imagenet", input_shape = (32, 32, 3))

# # freeze the base model
# for layer in base_model.layers:
#     layer.trainable = False

# construct the classifier
model = MobileNetClassifier.build(base_model, 10)

# compile the model
opt = SGD(lr = 0.01, momentum = 0.9, nesterov = True)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [ ]:
# initialize the training parameters
epochs = 5
bs = 128
steps_per_epoch = np.ceil(x_train.shape[0] / bs)
max_lr = 0.01

# initialize the one cycle scheduler
ocs = OneCycleScheduler(epochs, max_lr, steps_per_epoch)

In [ ]:
# train the model
model.fit(x_train, y_train, validation_data = (x_test, y_test), 
          epochs = epochs, batch_size = bs,
          callbacks = [ocs])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (8, 3))
ax[0].plot(ocs.history["lrs"])
ax[0].set_title("Learning Rate")
ax[0].set_xlabel("Iterations")
ax[1].plot(ocs.history["moms"])
ax[1].set_title("Momentum")
ax[1].set_xlabel("Iterations")
plt.show()